In [587]:
%matplotlib inline
from __future__ import division
import numpy as np
from numpy.random import rand
import matplotlib.pyplot as plt
import os
from matplotlib.colors import ListedColormap
import matplotlib.colors
import time

from PIL import Image, ImageDraw

In [588]:
def getValids(xsize, ysize):

  invalids = []
  valids = []
  for x in range(xsize):
    for y in range(ysize):
      a = 2
      if y % 2 == 1:
        a = 0

      if x % 3 == a: #doublecheck this
        invalids.append([x, y])
      else:
        valids.append([x, y])
  return [invalids, valids]

def maketrix(n):
  if n % 2 != 0:
    raise Exception('n must be even')
  
  xval = n-1 #must be odd
  yval = n 
  matrix = np.random.randint(1, size=(xval,yval)) #this is just all 0s side note what is this structure
  arrays = getValids(xval, yval)

  for xy in arrays[1]:
    #print(xy)
    matrix[xy[0],xy[1]] = np.random.choice([-1, 1])

  return [matrix, arrays[1]]

In [589]:
class Ising():
    def __init__(self, temp, n, msrmnt):
        self.temp = temp
        self.n = n
        self.msrmnt = msrmnt

    ''' Simulating the Ising model '''    
    ## monte carlo moves
    def mcmove(self, config, valids, N, beta):
        ''' This is to execute the monte carlo moves using 
        Metropolis algorithm such that detailed
        balance condition is satisified'''
        validslength = len(valids)
        xN = N-1
        randomStarts = np.random.randint(validslength, size = N*N)
        for i in range(N*N):
            [a, b] = valids[randomStarts[i]]

            s =  config[a, b]
            if s == 0:#REMOVE LATER
                print("there's a problem here")
            

            if b % 2 == 0:
                if a % 3 == 0:
                    nb = config[(a+1)%xN, b] + config[(a-1)%xN, (b-1)%N] + config[(a-1)%xN, (b+1)%N]
                else:
                    nb = config[(a-1)%xN, b] + config[a, (b-1)%N] + config[a, (b+1)%N]
            else:
                if a % 3 == 2:
                    nb = config[(a-1)%xN, b] + config[(a+1)%xN, (b-1)%N] + config[(a+1)%xN, (b+1)%N]
                else:
                    nb = config[(a+1)%xN, b] + config[a, (b-1)%N] + config[a, (b+1)%N]
            
            cost = 2*s*nb
            
            #if cost < 0: #believe this is extraneous
            #    s *= -1
            if rand() < np.exp(-cost*beta):
                s *= -1
            config[a, b] = s
        return config
    
    def simulate(self):   
        ''' This module simulates the Ising model'''
#***MODIFY THIS TO CONVERT self.temp TEMPERATURE to thermodynamic beta*** 
        temp  = self.temp
        N     = self.n        # Initialise the lattice
        msrmnt = self.msrmnt
        #config = 2*np.random.randint(2, size=(N,N))-1 #discard
        [config, valids] = maketrix(n)
        f = plt.figure(figsize=(15, 15), dpi=80);    
        #self.configPlot(f, config, 0, N, 1);
        # naming convention is path/Ising_<temperature>
        plotpath = ""
        plotname = [plotpath,"triIsing_",str(self.temp),"_"]
        plotname = "".join(plotname)
        i = 0

        folderprefix = "triangular-dataset/"
        folderprefix2 = str(self.temp) + "/"

        if not os.path.isdir(folderprefix):
            os.mkdir(folderprefix)
        
        folderprefix += folderprefix2

        if not os.path.isdir(folderprefix):
            os.mkdir(folderprefix)

        while os.path.exists(f"{folderprefix + plotname}{i}.png"):
            i += 1
        plotname = [plotname,str(i),".png"]
        print(plotname)
        plotname = "".join(plotname)
        
        for i in range(msrmnt+1):
            #start_time = time.time()
            self.mcmove(config, valids, N,  1.0/(temp))
            #end_time = time.time() - start_time
            #print("iteration: " + str(i) + " ---- time taken: " + str(end_time))
            if i == msrmnt:    
                self.configPlot(f, config, i, N, 1);
        
        
        
        f.savefig(folderprefix + plotname, transparent = True, bbox_inches = 'tight', pad_inches = 0)

        makeTriangleImage(N, config, plotname, folderprefix)
        plt.close(f)
                    
    def configPlot(self, f, config, i, N, n_):
        ''' This modules plts the configuration once passed to it along with time etc '''
        X, Y = np.meshgrid(range(N+1), range(N))
        sp =  f.add_subplot(3, 3, n_ )  
        plt.setp(sp.get_yticklabels(), visible=False)
        plt.setp(sp.get_xticklabels(), visible=False)      
        plt.gca().set_axis_off()
        plt.subplots_adjust(top = 1, bottom = 0, right = 1, left = 0, 
            hspace = 0, wspace = 0)
        plt.margins(0,0)
        #this is ridiculously suboptimal but i see no other way to do this with cmaps
        #extra N^2 operations for no real reason
        z = 0
        for i in range(N-1):
            for j in range(N):
                if config[i][j] != 1:
                    z = 1
        #REMOVE MAYBE
        #print(config)

        if z:
            plt.pcolormesh(X, Y, config, cmap=plt.cm.RdBu);
        else:
            bluemap = ListedColormap((0.019, 0.188, 0.380, 1))
            plt.pcolormesh(X, Y, config, cmap=bluemap);
    

    

In [590]:
from PIL import Image, ImageDraw, ImageOps

def makeTriangleImage(n, config, plotname, folderprefix): #make actually work lol
    img = Image.new("RGB", (2*n, 2*n-1))
    imred = Image.new("RGB", (2, 2), color = (103, 0, 31))
    imbl = Image.new("RGB", (2, 2), color = (5, 48, 97))
    imwh = Image.new("RGBA", (2, 2), color = (0, 0, 0, 0))

    #print(config)

    for x in range(n-1):
        for y in range(n):
            if config[x, y] == 1:
                im = imbl #blue is 1
            elif config[x, y] == -1:
                im = imred
            else:
                im = imwh

            if y % 2 == 0:
                img.paste(im, (2*y, 2*x))
            else:
                img.paste(im, (2*y, 2*x+1))
            
    img = ImageOps.flip(img)
    img.save(folderprefix+"adjusted"+plotname)


In [591]:
# temp_dist is the input temperature distribution
temp_dist = np.array([0.05, 0.1, 0.5, 1, 2, 3, 4, 5, 6, 10, 16, 20]) 
# n is resolution of image (n-1) x n
n = 64
# Final iteration/time
msrmnt = 30000

for x in range(3):
    for temp in temp_dist:
        rm = Ising(temp, n, msrmnt)
        rm.simulate()

print('completed')

['triIsing_0.05_', '0', '.png']
['triIsing_0.1_', '0', '.png']
['triIsing_0.5_', '0', '.png']
['triIsing_1.0_', '0', '.png']
['triIsing_2.0_', '0', '.png']
['triIsing_3.0_', '0', '.png']
['triIsing_4.0_', '0', '.png']
['triIsing_5.0_', '0', '.png']
['triIsing_6.0_', '0', '.png']
['triIsing_10.0_', '0', '.png']
['triIsing_16.0_', '0', '.png']
['triIsing_20.0_', '0', '.png']
['triIsing_0.05_', '1', '.png']
['triIsing_0.1_', '1', '.png']
['triIsing_0.5_', '1', '.png']
['triIsing_1.0_', '1', '.png']
['triIsing_2.0_', '1', '.png']
['triIsing_3.0_', '1', '.png']
['triIsing_4.0_', '1', '.png']
['triIsing_5.0_', '1', '.png']
['triIsing_6.0_', '1', '.png']
['triIsing_10.0_', '1', '.png']
['triIsing_16.0_', '1', '.png']
['triIsing_20.0_', '1', '.png']
['triIsing_0.05_', '2', '.png']
['triIsing_0.1_', '2', '.png']
['triIsing_0.5_', '2', '.png']
['triIsing_1.0_', '2', '.png']
['triIsing_2.0_', '2', '.png']
['triIsing_3.0_', '2', '.png']
['triIsing_4.0_', '2', '.png']
['triIsing_5.0_', '2', '.png']